In [5]:
import sys
libpath = "/home/pablo/tesis/tsm-pipeline/code"
if libpath not in sys.path:
    sys.path.append(libpath)
    
%load_ext autoreload
%autoreload 2

import docker
docker_client = docker.from_env()

# Configura todo lo necesario en el ambiente
from scripts.docker import run_tsm, ExperimentSettings, read_dbs_dataset
from scripts.evaluate import evaluate, EvaluationSettings
from os.path import join as path_join
import logging

logging.basicConfig(level=logging.INFO)

defaults = {
    "bash_config_path": "/home/pablo/tesis/tsm-pipeline/code/scripts/config.sh",
}
defaults_evalute = {
    "cache_root":"/home/pablo/dbcache",    
}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
nosql_dbs = read_dbs_dataset("/home/pablo/tesis/tsm-pipeline/experiments/datasets/nosql_1.txt")
from numpy.random import RandomState

# Using a fixed random state to have reproducible results
rand = RandomState(seed=5)

In [27]:
from sklearn.model_selection import train_test_split

sample = nosql_dbs[:100]
train, test = train_test_split(sample, train_size=.5, shuffle=True, random_state=rand)
train[:10]

['DBULL7/uhoops/ffd7a04',
 'bookbrainz/bookbrainz-site/d4381bb',
 'aali05/solo-project/3d3cf57',
 'Cendey/contacts/252de32',
 'FreetailHackers/saffron/f563dd2',
 'StephenGrider/AdvancedReduxCode/9f839ad',
 'ForestAdmin/forest-express-mongoose/2b9166b',
 'bailicangdu/node-elm/4262918',
 'MauroEspinosa/Grupos/7ac4ea1',
 'FSecureLABS/dref/c4e5216']

Primera corrida de bejamin button, con un dataset formado a partir de repos que venian con commits, y filtrando que tengan por lo menos 5k loc de javascript. Se toma un corte de 50/50 entre entrenamiento y test.

In [10]:
results_dir = "/home/pablo/results/ds_nosql_1"

In [11]:
# training
train_settings = ExperimentSettings(
    name="ds_nosql_1",
    query_type="nosql",
    project_list=train,
    results_dir=results_dir,
    **defaults
)

run_tsm(docker_client, train_settings, block=True)

running at container f6e21cff2d3ec89b824fe4805adbdb3e6394f2b1795a94d8bbb39cfc8c00da3a. Use `docker logs f6e21cff2d3ec89b824fe4805adbdb3e6394f2b1795a94d8bbb39cfc8c00da3a --tail 10 --follow` to follow progress
g: /home/pablo/results/ds_nosql_1/*/NosqlInjectionWorse-*/reprScores.txt


In [12]:
!cat /home/pablo/results/ds_nosql_1/averaged-results.csv|wc -l

51


In [13]:
test[0:10]

['PaulSec/VNWA/d3704e6',
 'FreetailHackers/saffron/f563dd2',
 'JosipReh/Feeddit/bbcb615',
 'EQuimper/nodejs-api-boilerplate/0dcea20',
 'Xerios/mobx-isomorphic-starter/8d08656',
 'DanialK/PassportJS-Authentication/af6f691',
 'BryanKo/Fam/6ee0fe0',
 'DanWahlin/Angular-RESTfulService/bbd42e4',
 'Crizstian/cinema-microservice/b6c973d',
 'Gingernaut/Robot-Lawyer/56b86ac']

In [15]:
# evaluation
external_predicate_file = path_join(results_dir, 'averaged-results.csv')
worse_settings = EvaluationSettings(
    search_path="/home/pablo/tesis/tsm-pipeline/lib-worse/codeql/javascript/ql:/home/pablo/.codeql/packages/codeql/javascript-upgrades/0.0.3",
    cli_version="2.13.1",
    db_cli_version="2.5.2",
    query_file="/home/pablo/tesis/tsm-pipeline/tsm-atm-pipeline/src/tsm/evaluation/NosqlInjectionWorseTSM.ql",
    external_predicate_file=external_predicate_file,
    **defaults_evalute,
)

print("Evaluating worse with %d dbs" % (len(test)))
evaluate(
    settings=worse_settings,
    output_dir=path_join(results_dir, 'worse'),
    dbs=test,
)

INFO:scripts.evaluate:creating /home/pablo/results/ds_nosql_1/worse dir. Omitting error if already existing


Evaluating worse with 50 dbs


INFO:scripts.evaluate:Starting processing with 4 processes
INFO:scripts.evaluate:Evaluating Xerios - mobx-isomorphic-starter
INFO:scripts.evaluate:Evaluating PaulSec - VNWA
INFO:scripts.evaluate:Evaluating Crizstian - cinema-microservice
INFO:scripts.evaluate:Evaluating StudioDeTelevision - admiraljs
INFO:scripts.evaluate:Evaluating FreetailHackers - saffron
INFO:scripts.evaluate:Evaluating Gingernaut - Robot-Lawyer
INFO:scripts.evaluate:Evaluating DanialK - PassportJS-Authentication
INFO:scripts.evaluate:Evaluating IBM - innovate-digital-bank
INFO:scripts.evaluate:Evaluating bitpay - bitcore
INFO:scripts.evaluate:Evaluating BryanKo - Fam
INFO:scripts.evaluate:Evaluating JosipReh - Feeddit
INFO:scripts.evaluate:Evaluating OmgImAlexis - astro
INFO:scripts.evaluate:Evaluating DanWahlin - Angular-RESTfulService
INFO:scripts.evaluate:Evaluating EQuimper - nodejs-api-boilerplate
INFO:scripts.evaluate:Evaluating FrontendMasters - intro-mongo-db
INFO:scripts.evaluate:Evaluating IHTSDO - sct-s

In [16]:
v0_settings = EvaluationSettings(
    search_path="/home/pablo/codeqlv0/javascript/ql/lib",
    cli_version="2.13.1",
    db_cli_version="2.5.2",
    query_file="/home/pablo/tesis/tsm-pipeline/tsm-evaluation/tsm-evaluation/src/NoSqlEvaluation.ql",
    **defaults_evalute,
)

print("Evaluating v0 with %d dbs" % (len(test)))
evaluate(
    settings=v0_settings,
    output_dir=path_join(results_dir, 'v0'),
    dbs=test,
)

INFO:scripts.evaluate:creating /home/pablo/results/ds_nosql_1/v0 dir. Omitting error if already existing


Evaluating v0 with 50 dbs


INFO:scripts.evaluate:Starting processing with 4 processes
INFO:scripts.evaluate:Evaluating PaulSec - VNWA
INFO:scripts.evaluate:Evaluating Crizstian - cinema-microservice
INFO:scripts.evaluate:Evaluating StudioDeTelevision - admiraljs
INFO:scripts.evaluate:Evaluating Xerios - mobx-isomorphic-starter
INFO:scripts.evaluate:Evaluating Gingernaut - Robot-Lawyer
INFO:scripts.evaluate:Evaluating DanialK - PassportJS-Authentication
INFO:scripts.evaluate:Evaluating FreetailHackers - saffron
INFO:scripts.evaluate:Evaluating IBM - innovate-digital-bank
INFO:scripts.evaluate:Evaluating BryanKo - Fam
INFO:scripts.evaluate:Evaluating bitpay - bitcore
INFO:scripts.evaluate:Evaluating JosipReh - Feeddit
INFO:scripts.evaluate:Evaluating OmgImAlexis - astro
INFO:scripts.evaluate:Evaluating DanWahlin - Angular-RESTfulService
INFO:scripts.evaluate:Evaluating EQuimper - nodejs-api-boilerplate
INFO:scripts.evaluate:Evaluating FrontendMasters - intro-mongo-db
INFO:scripts.evaluate:Evaluating IHTSDO - sct-s

In [24]:
from scripts.calculate_scores import calculate_scores_df

calculate_scores_df(
    results_folder=results_dir,
    cleanup_base_dir="/tmp",
)

,precision,recall,accuracy,alerts to recover (atr),alerts recovered,suprious alerts,projects with atr,avg atr per proj
0,0.0,0.0,0.0,21,0,306,8,2.625


## Tomo otro corte diferente el mismo sample averr si cambia algo

In [34]:
train2, test2 = train_test_split(sample, train_size=.5, shuffle=True, random_state=rand)
train2[:10]

['OpenUserJS/OpenUserJS.org/a3728c9',
 'Redsmin/proxy/0b20436',
 'abansal98/DataPool/5d0aeb7',
 'DimiMikadze/Mean-Blog/01c6ff8',
 'bdvu/BVmeanauthapp/5521a12',
 'ParabolInc/action/45c9b49',
 'StephenGrider/AdvancedReduxCode/9f839ad',
 'aheckmann/mquery/e68f8e1',
 'bergwhite/nchat/5442684',
 'F0xedb/WebCalendar/a18c0a0']

In [1]:
results_dir = "/home/pablo/results/ds_nosql_2"

In [36]:
# training
train_settings = ExperimentSettings(
    name="ds_nosql_2",
    query_type="nosql",
    project_list=train2,
    results_dir=results_dir,
    **defaults
)

run_tsm(docker_client, train_settings, block=True)

INFO:scripts.docker:creating /home/pablo/results/ds_nosql_2 dir. Omitting error if already existing
INFO:scripts.docker:creating /tmp/log dir. Omitting error if already existing
INFO:scripts.docker:running at container 2ae4661619bdb52ca4625fee3c8d3d01f57a9836a75763340da7851a6d085aec. Use `docker logs 2ae4661619bdb52ca4625fee3c8d3d01f57a9836a75763340da7851a6d085aec --tail 10 --follow` to follow progress


running at container 2ae4661619bdb52ca4625fee3c8d3d01f57a9836a75763340da7851a6d085aec. Use `docker logs 2ae4661619bdb52ca4625fee3c8d3d01f57a9836a75763340da7851a6d085aec --tail 10 --follow` to follow progress


INFO:scripts.docker:running combine scores


g: /home/pablo/results/ds_nosql_2/*/NosqlInjectionWorse-*/reprScores.txt


In [37]:
!cat /home/pablo/results/ds_nosql_2/averaged-results.csv|wc -l

52


In [38]:
test2[0:10]

['Bertrand31/Monitaure/edbc31b',
 'HashBrownCMS/hashbrown-cms/b987436',
 'CapacitorSet/rebridge/451bfc2',
 'aali05/solo-project/3d3cf57',
 'LiangCY/MovieTickets/0069729',
 'TylerBrock/mongo-hacker/6a5a9f4',
 'JPinkney/Track-your-TV/e09bcfa',
 'BlinkTagInc/gtfs-to-html/ddb81fe',
 'AdoboFrenzy/Projects/9f25394',
 'amejiarosario/meanshop/305a189']

In [39]:
# evaluation
external_predicate_file = path_join(results_dir, 'averaged-results.csv')
worse_settings = EvaluationSettings(
    search_path="/home/pablo/tesis/tsm-pipeline/lib-worse/codeql/javascript/ql:/home/pablo/.codeql/packages/codeql/javascript-upgrades/0.0.3",
    cli_version="2.13.1",
    db_cli_version="2.5.2",
    query_file="/home/pablo/tesis/tsm-pipeline/tsm-atm-pipeline/src/tsm/evaluation/NosqlInjectionWorseTSM.ql",
    external_predicate_file=external_predicate_file,
    **defaults_evalute,
)

evaluate(
    settings=worse_settings,
    output_dir=path_join(results_dir, 'worse'),
    dbs=test2,
)

INFO:scripts.evaluate:creating /home/pablo/results/ds_nosql_2/worse dir. Omitting error if already existing
INFO:scripts.evaluate:Starting processing with 4 processes
INFO:scripts.evaluate:Evaluating Bertrand31 - Monitaure
INFO:scripts.evaluate:Evaluating DanWahlin - CustomerManagerStandard
INFO:scripts.evaluate:Evaluating LiangCY - MovieTickets
INFO:scripts.evaluate:Evaluating AdoboFrenzy - Projects
INFO:scripts.evaluate:Evaluating TylerBrock - mongo-hacker
INFO:scripts.evaluate:Evaluating HashBrownCMS - hashbrown-cms
INFO:scripts.evaluate:Evaluating DrMongo - DrMongo
INFO:scripts.evaluate:Evaluating amejiarosario - meanshop
INFO:scripts.evaluate:Evaluating JPinkney - Track-your-TV
INFO:scripts.evaluate:Evaluating PaulSec - VNWA
INFO:scripts.evaluate:Evaluating CapacitorSet - rebridge
INFO:scripts.evaluate:Evaluating BlinkTagInc - node-gtfs
INFO:scripts.evaluate:Evaluating BlinkTagInc - gtfs-to-html
INFO:scripts.evaluate:Evaluating Gingernaut - Robot-Lawyer
INFO:scripts.evaluate:Evalu

In [40]:
v0_settings = EvaluationSettings(
    search_path="/home/pablo/codeqlv0/javascript/ql/lib",
    cli_version="2.13.1",
    db_cli_version="2.5.2",
    query_file="/home/pablo/tesis/tsm-pipeline/tsm-evaluation/tsm-evaluation/src/NoSqlEvaluation.ql",
    **defaults_evalute,
)

print("Evaluating v0 with %d dbs" % (len(test)))
evaluate(
    settings=v0_settings,
    output_dir=path_join(results_dir, 'v0'),
    dbs=test2,
)

INFO:scripts.evaluate:creating /home/pablo/results/ds_nosql_2/v0 dir. Omitting error if already existing


Evaluating v0 with 50 dbs


INFO:scripts.evaluate:Starting processing with 4 processes
INFO:scripts.evaluate:Evaluating AdoboFrenzy - Projects
INFO:scripts.evaluate:Evaluating LiangCY - MovieTickets
INFO:scripts.evaluate:Evaluating DanWahlin - CustomerManagerStandard
INFO:scripts.evaluate:Evaluating Bertrand31 - Monitaure
INFO:scripts.evaluate:Evaluating TylerBrock - mongo-hacker
INFO:scripts.evaluate:Evaluating DrMongo - DrMongo
INFO:scripts.evaluate:Evaluating HashBrownCMS - hashbrown-cms
INFO:scripts.evaluate:Evaluating JPinkney - Track-your-TV
INFO:scripts.evaluate:Evaluating PaulSec - VNWA
INFO:scripts.evaluate:Evaluating amejiarosario - meanshop
INFO:scripts.evaluate:Evaluating CapacitorSet - rebridge
INFO:scripts.evaluate:Evaluating BlinkTagInc - gtfs-to-html
INFO:scripts.evaluate:Evaluating aali05 - solo-project
INFO:scripts.evaluate:Evaluating Gingernaut - Robot-Lawyer
INFO:scripts.evaluate:Evaluating BlinkTagInc - node-gtfs
INFO:scripts.evaluate:Evaluating WaftTech - WaftEngine
INFO:scripts.evaluate:Eva

In [41]:
from scripts.calculate_scores import calculate_scores_df

calculate_scores_df(
    results_folder=results_dir,
    cleanup_base_dir="/tmp",
)

,precision,recall,accuracy,alerts to recover (atr),alerts recovered,suprious alerts,projects with atr,avg atr per proj
0,0.0,0.0,0.0,84,0,297,10,8.4


Lei en unas slides de diego que para evaluar, usan como v0 la ultima version. Probando con el ultimo commit aver como funciona, guardado en `/home/pablo/codeql_experimentation/javascript/ql/lib`. Freeznadola al ultimo release estable: `c1a52031cfd323b34a039cc6dcda4f73d7ad892e`, que corresponde con 2.13.1

In [ ]:
v0_experiment_settings = EvaluationSettings(
    search_path="/home/pablo/codeql_experimentation/javascript/ql/lib",
    cli_version="2.13.1",
    db_cli_version="2.5.2",
    query_file="/home/pablo/tesis/tsm-pipeline/tsm-evaluation/tsm-evaluation/src/NoSqlEvaluation.ql",
    **defaults_evalute,
)

print("Evaluating v0 with latest codeql commit with %d dbs" % (len(test)))
evaluate(
    settings=v0_experiment_settings,
    output_dir=path_join(results_dir, 'v0_experiment'),
    dbs=test2,
)

INFO:scripts.evaluate:creating /home/pablo/results/ds_nosql_2/v0_experiment dir. Omitting error if already existing


Evaluating v0 with latest codeql commit with 50 dbs


INFO:scripts.evaluate:Starting processing with 4 processes
INFO:scripts.evaluate:Evaluating LiangCY - MovieTickets
INFO:scripts.evaluate:Evaluating AdoboFrenzy - Projects
INFO:scripts.evaluate:Evaluating DanWahlin - CustomerManagerStandard
INFO:scripts.evaluate:Evaluating Bertrand31 - Monitaure


In [4]:
from scripts.calculate_scores import calculate_scores_df, _calculate_score_sets, hash_set_to_df

calculate_scores_df(
    results_folder=results_dir,
    cleanup_base_dir="/tmp",
    v0_dir="v0_experiment",
)

# v0, worse, boosted = _calculate_score_sets(
#     results_folder=results_dir,
#     cleanup_base_dir="/tmp",
#     v0_dir="v0_experiment",
# )

# hash_set_to_df(v0-worse)

,precision,recall,accuracy,alerts to recover (atr),alerts recovered,suprious alerts,projects with atr,avg atr per proj
0,0.0,0.0,0.0,84,0,297,10,8.4


In [53]:
!cat $results_dir/averaged-results.txt

module TsmRepr {
  float getReprScore(string repr, string t){
   repr = "(parameter 0 (member AchievementUnlock *))" and t = "snk" and result = 0.2500000000 or
   repr = "(parameter 0 (member Editor *))" and t = "snk" and result = 0.2500000000 or
   repr = "(parameter 0 (member ObjectID *))" and t = "snk" and result = 0.2500000000 or
   repr = "(parameter 0 (member _verifySignature *))" and t = "snk" and result = 0.2500000000 or
   repr = "(parameter 0 (member broadcast *))" and t = "snk" and result = 0.2500000000 or
   repr = "(parameter 0 (member count *))" and t = "snk" and result = 1.0000000000 or
   repr = "(parameter 0 (member countDocuments *))" and t = "snk" and result = 0.2500000000 or
   repr = "(parameter 0 (member deleteOne *))" and t = "snk" and result = 0.2500000000 or
   repr = "(parameter 0 (member differenceBy *))" and t = "snk" and result = 0.2500000000 or
   repr = "(parameter 0 (member duration *))" and t = "snk" and result = 0.2500000000 or
   repr = "(parameter 0 

## Corriendo con 200 dbs

In [7]:
from sklearn.model_selection import train_test_split

sample = nosql_dbs[:200]
train, test = train_test_split(sample, train_size=.5, shuffle=True, random_state=rand)
train[:10]

['Gingernaut/Robot-Lawyer/56b86ac',
 'StudioDeTelevision/admiraljs/5846378',
 'aheckmann/mquery/e68f8e1',
 'bailicangdu/node-elm/4262918',
 'bitpay/bitcore/4f8f5a2',
 'aravindvnair99/Prostem-Internship/a0ecb39',
 'F0xedb/WebCalendar/a18c0a0',
 'hex7c0/mongodb-backup/b13dd25',
 'OmgImAlexis/astro/d650f5e',
 'eddywashere/node-stripe-membership-saas/206afec']

Primera corrida de bejamin button, con un dataset formado a partir de repos que venian con commits, y filtrando que tengan por lo menos 5k loc de javascript. Se toma un corte de 50/50 entre entrenamiento y test.

In [16]:
results_dir = "/home/pablo/results/ds_nosql_200"

In [11]:
# training
train_settings = ExperimentSettings(
    name="ds_nosql_100",
    query_type="nosql",
    project_list=train,
    results_dir=results_dir,
    **defaults
)

run_tsm(docker_client, train_settings, block=True)

running at container f6e21cff2d3ec89b824fe4805adbdb3e6394f2b1795a94d8bbb39cfc8c00da3a. Use `docker logs f6e21cff2d3ec89b824fe4805adbdb3e6394f2b1795a94d8bbb39cfc8c00da3a --tail 10 --follow` to follow progress
g: /home/pablo/results/ds_nosql_1/*/NosqlInjectionWorse-*/reprScores.txt


In [12]:
!cat $results_dir/averaged-results.csv|wc -l

51


In [13]:
test[0:10]

['PaulSec/VNWA/d3704e6',
 'FreetailHackers/saffron/f563dd2',
 'JosipReh/Feeddit/bbcb615',
 'EQuimper/nodejs-api-boilerplate/0dcea20',
 'Xerios/mobx-isomorphic-starter/8d08656',
 'DanialK/PassportJS-Authentication/af6f691',
 'BryanKo/Fam/6ee0fe0',
 'DanWahlin/Angular-RESTfulService/bbd42e4',
 'Crizstian/cinema-microservice/b6c973d',
 'Gingernaut/Robot-Lawyer/56b86ac']

In [15]:
# evaluation
external_predicate_file = path_join(results_dir, 'averaged-results.csv')
worse_settings = EvaluationSettings(
    search_path="/home/pablo/tesis/tsm-pipeline/lib-worse/codeql/javascript/ql:/home/pablo/.codeql/packages/codeql/javascript-upgrades/0.0.3",
    cli_version="2.13.1",
    db_cli_version="2.5.2",
    query_file="/home/pablo/tesis/tsm-pipeline/tsm-atm-pipeline/src/tsm/evaluation/NosqlInjectionWorseTSM.ql",
    external_predicate_file=external_predicate_file,
    **defaults_evalute,
)

print("Evaluating worse with %d dbs" % (len(test)))
evaluate(
    settings=worse_settings,
    output_dir=path_join(results_dir, 'worse'),
    dbs=test,
)

INFO:scripts.evaluate:creating /home/pablo/results/ds_nosql_1/worse dir. Omitting error if already existing


Evaluating worse with 50 dbs


INFO:scripts.evaluate:Starting processing with 4 processes
INFO:scripts.evaluate:Evaluating Xerios - mobx-isomorphic-starter
INFO:scripts.evaluate:Evaluating PaulSec - VNWA
INFO:scripts.evaluate:Evaluating Crizstian - cinema-microservice
INFO:scripts.evaluate:Evaluating StudioDeTelevision - admiraljs
INFO:scripts.evaluate:Evaluating FreetailHackers - saffron
INFO:scripts.evaluate:Evaluating Gingernaut - Robot-Lawyer
INFO:scripts.evaluate:Evaluating DanialK - PassportJS-Authentication
INFO:scripts.evaluate:Evaluating IBM - innovate-digital-bank
INFO:scripts.evaluate:Evaluating bitpay - bitcore
INFO:scripts.evaluate:Evaluating BryanKo - Fam
INFO:scripts.evaluate:Evaluating JosipReh - Feeddit
INFO:scripts.evaluate:Evaluating OmgImAlexis - astro
INFO:scripts.evaluate:Evaluating DanWahlin - Angular-RESTfulService
INFO:scripts.evaluate:Evaluating EQuimper - nodejs-api-boilerplate
INFO:scripts.evaluate:Evaluating FrontendMasters - intro-mongo-db
INFO:scripts.evaluate:Evaluating IHTSDO - sct-s

In [16]:
v0_settings = EvaluationSettings(
    search_path="/home/pablo/codeqlv0/javascript/ql/lib",
    cli_version="2.13.1",
    db_cli_version="2.5.2",
    query_file="/home/pablo/tesis/tsm-pipeline/tsm-evaluation/tsm-evaluation/src/NoSqlEvaluation.ql",
    **defaults_evalute,
)

print("Evaluating v0 with %d dbs" % (len(test)))
evaluate(
    settings=v0_settings,
    output_dir=path_join(results_dir, 'v0'),
    dbs=test,
)

INFO:scripts.evaluate:creating /home/pablo/results/ds_nosql_1/v0 dir. Omitting error if already existing


Evaluating v0 with 50 dbs


INFO:scripts.evaluate:Starting processing with 4 processes
INFO:scripts.evaluate:Evaluating PaulSec - VNWA
INFO:scripts.evaluate:Evaluating Crizstian - cinema-microservice
INFO:scripts.evaluate:Evaluating StudioDeTelevision - admiraljs
INFO:scripts.evaluate:Evaluating Xerios - mobx-isomorphic-starter
INFO:scripts.evaluate:Evaluating Gingernaut - Robot-Lawyer
INFO:scripts.evaluate:Evaluating DanialK - PassportJS-Authentication
INFO:scripts.evaluate:Evaluating FreetailHackers - saffron
INFO:scripts.evaluate:Evaluating IBM - innovate-digital-bank
INFO:scripts.evaluate:Evaluating BryanKo - Fam
INFO:scripts.evaluate:Evaluating bitpay - bitcore
INFO:scripts.evaluate:Evaluating JosipReh - Feeddit
INFO:scripts.evaluate:Evaluating OmgImAlexis - astro
INFO:scripts.evaluate:Evaluating DanWahlin - Angular-RESTfulService
INFO:scripts.evaluate:Evaluating EQuimper - nodejs-api-boilerplate
INFO:scripts.evaluate:Evaluating FrontendMasters - intro-mongo-db
INFO:scripts.evaluate:Evaluating IHTSDO - sct-s

In [17]:
from scripts.calculate_scores import calculate_scores_df

calculate_scores_df(
    results_folder=results_dir,
    cleanup_base_dir="/tmp",
)

,precision,recall,accuracy,alerts to recover (atr),alerts recovered,suprious alerts,projects with atr,avg atr per proj
0,0.0,0.0,0.0,50,0,952,15,3.333333
